# Reward test
Let's see if buying the asset for a discounted price or selling it for a slightly higher price actually look profitable from the perspective of the discounted future reward.

In [1]:
import numpy as np
import plotly.graph_objects as go

In [2]:
discount_factor = 0.99
period = 16
start_cash = 1024
start_asset = 1024
buy_price = 1 / 1.1
sell_price = 1.1
price_process = lambda step: np.exp(0.25 * np.sin(step * 2 * np.pi / period))
utility = lambda cash, asset, step: np.log(1 + cash + price_process(step) * asset)

def evaluation(cash, asset, step):
    offsets = np.arange(65536)
    return np.sum(utility(cash, asset, step + offsets) * discount_factor ** offsets)

steps = np.arange(256)
prices = price_process(steps)
go.Figure(
    layout=dict(
        xaxis_title="Asset price",
        yaxis_title="Reward",
    ),
    data=[
        go.Scatter(
            name="No action",
            x=prices,
            y=[evaluation(cash=start_cash, asset=start_asset, step=step) for step in steps],
        ),
        go.Scatter(
            name="After purchase",
            x=prices,
            y=[evaluation(cash=start_cash - buy_price, asset=start_asset + 1, step=step) for step in steps],
        ),
        go.Scatter(
            name="After sale",
            x=prices,
            y=[evaluation(cash=start_cash + sell_price, asset=start_asset - 1, step=step) for step in steps],
        ),
    ]
)

It turns out that the expected future utility depends much more on where we are on this circle than on what action we took. Probably a really bad sign!

Incorporating history might help a bit so that the critic knows where on this circle we are. More importantly, though, we need a reward signal which does not treat actions with so much indifference!

In [4]:
utility = lambda cash, asset, step: np.log(1 + cash + price_process(step) * asset)
delta_reward = lambda cash, asset, step, next_cash, next_asset: utility(
    cash=next_cash, asset=next_asset, step=step + 1
) - utility(cash=cash, asset=asset, step=step)


def delta_evaluation(cash, asset, step, next_cash, next_asset):
    offsets = np.arange(65536)
    return np.sum(
        delta_reward(cash, asset, step + offsets, next_cash, next_asset)
        * discount_factor ** offsets
    )


go.Figure(
    layout=dict(
        xaxis_title="Asset price",
        yaxis_title="Reward",
    ),
    data=[
        go.Scatter(
            name="No action",
            x=prices,
            y=[
                delta_evaluation(
                    cash=start_cash,
                    asset=start_asset,
                    step=step,
                    next_cash=start_cash,
                    next_asset=start_asset,
                )
                for step in steps
            ],
        ),
        go.Scatter(
            name="Purchase",
            x=prices,
            y=[
                delta_evaluation(
                    cash=start_cash,
                    asset=start_asset,
                    step=step,
                    next_cash=start_cash - buy_price,
                    next_asset=start_asset + 1,
                )
                for step in steps
            ],
        ),
        go.Scatter(
            name="Sale",
            x=prices,
            y=[
                delta_evaluation(
                    cash=start_cash,
                    asset=start_asset,
                    step=step,
                    next_cash=start_cash + sell_price,
                    next_asset=start_asset - 1,
                )
                for step in steps
            ],
        ),
    ],
)


When we use the change of utility as a reward, the relative effect of actions is larger, but still not all that large.